In [ ]:
import pandas as pd
import geopandas as gpd

data = pd.read_csv('E:\\fl_leuk\\health\\cc-est2023-agesex-12.csv')

# Calculate the total population for 0-15 years
age_0_15 = data['UNDER5_TOT'] + data['AGE513_TOT']
data['POP_0_15'] = age_0_15

age_65_plus = data['AGE65PLUS_TOT']
data['POP_65_PLUS'] = age_65_plus

# Select the geographic identifier columns and the new calculated columns
data['PCT_ATRISK'] = ((data['POP_0_15'] + data['POP_65_PLUS']) / data['POPESTIMATE'] * 100).round(2)
# Select the geographic identifier columns and the new calculated column
columns_to_keep = ['COUNTY', 'STNAME', 'CTYNAME', 'YEAR', 'PCT_ATRISK']
data = data[data['YEAR'] == 5]
data['CTYNAME'] = data['CTYNAME'].str.replace(' County', '')
result = data[columns_to_keep]
print(result.iloc[:, :5].head())
# Save the result to a new CSV file
result.to_csv('E:\\fl_leuk\\health\\atriskpop.csv', index=False)


In [ ]:
data2 = pd.read_csv('E:\\fl_leuk\\health\\obese.csv')
data3 = pd.read_csv('E:\\fl_leuk\\health\\pct_smokers.csv')
data2.info()
data3.info()

geodata  = gpd.read_file('E:\\fl_leuk\\leukemia_fl_updated.shp')
geodata.info()


In [ ]:
obesity_data = pd.read_csv('E:\\fl_leuk\\health\\obese.csv')
smoking_data = pd.read_csv('E:\\fl_leuk\\health\\pct_smokers.csv')

# Load GeoDataFrame
geodata = gpd.read_file('E:\\fl_leuk\\leukemia_fl_updated.shp')

# Merge Percent columns with GeoDataFrame
geodata = geodata.merge(obesity_data[['Name', 'obs_pct']], left_on='NAME', right_on='Name', how='left')
geodata = geodata.merge(smoking_data[['Name', 'Percent_smokers']], left_on='NAME', right_on='Name', how='left')
# Convert COUNTY in both datasets to string for consistent data type
result['COUNTY'] = result['COUNTY'].astype(str)
geodata['COUNTY'] = geodata['COUNTY'].astype(str)
# Merge PCT_ATRISK with GeoDataFrame
geodata = geodata.merge(result[['CTYNAME', 'PCT_ATRISK']], left_on='NAME', right_on='CTYNAME', how='left')



# Drop duplicate 'Name' columns after merge
geodata = geodata.drop(columns=['Name_x', 'Name_y','CTYNAME','PERCENT_AD'], errors='ignore')

# Save updated GeoDataFrame


In [59]:
data4 = pd.read_csv('E:\\fl_leuk\\hydrocarb_area_frac.csv')
data5 = pd.read_csv('E:\\fl_leuk\\pesticide_area_frac1.csv')

geodata['FIPS'] = geodata['FIPS'].astype(str)
data4['fips'] = data4['fips'].astype(str)
data5['fips'] = data5['fips'].astype(str)

geodata['FIPS'] = geodata['FIPS'].str.zfill(3)
data4['fips'] = data4['fips'].str.zfill(3)
data5['fips'] = data5['fips'].str.zfill(3)
# Merge hydrocarbon percentage
geodata = geodata.merge(data5[['fips', 'pesticide_pct']], 
                        left_on='FIPS', 
                        right_on='fips', 
                        how='left')

geodata = geodata.merge(data4[['fips', 'hydrocarbon_percentage']], 
                        left_on='FIPS', 
                        right_on='fips', 
                        how='left')
geodata.rename(columns={'hydrocarbon_percentage': 'hydro_pct'}, inplace=True)
# Optionally, drop the redundant fips columns if you want
geodata.drop(columns=['fips_x', 'fips_y'], inplace=True)
# Drop duplicate fips columns if created
geodata = geodata.drop(columns=['fips_x', 'fips_y'], errors='ignore')
geodata.to_file('E:\\fl_leuk\\leukemia_fl.shp')
geodata.head(n=10)

C:\Users\Andy\AppData\Local\Temp\ipykernel_20128\2277975463.py:26: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  geodata.to_file('E:\\fl_leuk\\leukemia_fl.shp')
d:\Users\Andy\miniconda3\envs\my_env\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Percent_smokers' to 'Percent_sm'
  ogr_write(
d:\Users\Andy\miniconda3\envs\my_env\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'pesticide_pct' to 'pesticide_'
  ogr_write(


,NAME,FIPS,INCIDENCE_,AVG_ANNUAL,cen_x,cen_y,COUNTY,POPESTIMAT,area_sq_km,ppl_per_sq,geometry,obs_pct,Percent_smokers,PCT_ATRISK,pesticide_pct,hydro_pct
0,Brevard,009,21.1,196,-8.987088e+06,3.286203e+06,009,643979,4341.836676,148,"POLYGON ((-9.02e+06 3.33e+06, -9.02e+06 3.33e+...",30.7,16.3,38.33,12.79,12.84
1,Columbia,023,17.7,17,-9.197380e+06,3.532487e+06,023,73063,2789.144396,26,"POLYGON ((-9.22e+06 3.5e+06, -9.22e+06 3.5e+06...",35.2,23.9,37.41,8.77,4.53
2,Gadsden,039,14.1,9,-9.419146e+06,3.578267e+06,039,43833,1853.111565,24,"POLYGON ((-9.45e+06 3.58e+06, -9.45e+06 3.58e+...",48.1,15.1,36.98,4.74,13.26
3,Highlands,055,14.9,30,-9.054843e+06,3.166474e+06,055,107614,3645.974628,30,"POLYGON ((-9.08e+06 3.17e+06, -9.08e+06 3.18e+...",29.1,16.5,48.61,37.84,16.96
4,Jefferson,065,0.0,3 or fewer,-9.339170e+06,3.559953e+06,065,15450,2142.135641,7,"POLYGON ((-9.36e+06 3.52e+06, -9.36e+06 3.54e+...",29.1,17.1,37.97,7.16,8.11
5,Leon,073,14.9,44,-9.381763e+06,3.562588e+06,073,296913,2454.205165,121,"POLYGON ((-9.43e+06 3.55e+06, -9.43e+06 3.55e+...",27.5,14.3,29.18,2.24,12.77
6,Orange,095,14.9,208,-9.052901e+06,3.314009e+06,095,1471416,3377.074532,436,"POLYGON ((-9.09e+06 3.35e+06, -9.09e+06 3.35e+...",31.2,11.7,29.64,11.04,22.36
7,Putnam,107,14.0,15,-9.099739e+06,3.453373e+06,107,75955,2843.097165,27,"POLYGON ((-9.13e+06 3.46e+06, -9.13e+06 3.46e+...",39.5,26.9,40.02,6.96,10.65
8,Taylor,123,13.7,4,-9.306715e+06,3.509643e+06,123,21582,3653.727409,6,"MULTIPOLYGON (((-9.33e+06 3.5e+06, -9.33e+06 3...",37.6,23.8,38.27,1.76,7.67
9,Volusia,127,17.5,149,-9.037151e+06,3.383137e+06,127,590357,4307.068308,137,"POLYGON ((-9.09e+06 3.42e+06, -9.09e+06 3.42e+...",35.5,18.5,38.83,5.68,13.80


In [60]:
geodata.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   NAME             67 non-null     object  
 1   FIPS             67 non-null     object  
 2   INCIDENCE_       67 non-null     float64 
 3   AVG_ANNUAL       67 non-null     object  
 4   cen_x            67 non-null     float64 
 5   cen_y            67 non-null     float64 
 6   COUNTY           67 non-null     object  
 7   POPESTIMAT       67 non-null     int64   
 8   area_sq_km       67 non-null     float64 
 9   ppl_per_sq       67 non-null     int32   
 10  geometry         67 non-null     geometry
 11  obs_pct          67 non-null     float64 
 12  Percent_smokers  67 non-null     float64 
 13  PCT_ATRISK       67 non-null     float64 
 14  pesticide_pct    67 non-null     float64 
 15  hydro_pct        67 non-null     float64 
dtypes: float64(9), geometry(1), int32(1), 